### Tabular Playground with PyTorch 
I this notebook I have attempted to use PyTorch to get the best result, so far I have not managed to beat score obtained with Tree regressors, but it is a work in progress. 


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%config Completer.use_jedi = False

In [ ]:
import category_encoders as ce
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler, QuantileTransformer

### Import the data. 

In [ ]:
cat_features = ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
numerical_features = ['cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5','cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13']

In [ ]:
df   = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/test.csv')
y = df.target

### Preprocessing 
In the function below I have done some simple preprocessing, scaling the numerical data and one hot encoding the catergorical data.  

In [ ]:
oh = ce.OneHotEncoder()
ss = QuantileTransformer()
def pre_pipe(fr, train=True):
    if train:
        c = fr[cat_features]
        c = oh.fit_transform(c)
        n = fr[numerical_features]
        n = pd.DataFrame(ss.fit_transform(n), columns=numerical_features)
        f = c.merge(n, left_index=True, right_index=True)
        return f
    else: 
        c = fr[cat_features]
        c = oh.transform(c)
        n = fr[numerical_features]
        n = pd.DataFrame(ss.transform(n), columns=numerical_features)
        f = c.merge(n, left_index=True, right_index=True)
        return f

In [ ]:
len(pre_pipe(df).columns), len(pre_pipe(test, False).columns)

In [ ]:
inp = 70
hid = 440
hid2 = 530
hid3 = 378
hid4 = 137
hid5 = 52

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, num_hidden):
        super().__init__()
        self.linear1    = nn.Linear(input_dim, num_hidden)
        self.linear2    = nn.Linear(num_hidden, hid2)
        self.linear3    = nn.Linear(hid2,hid3)
        self.linear4    = nn.Linear(hid3,hid4)
        self.linear5    = nn.Linear(hid4,hid5)
        self.batchNorm1 = nn.BatchNorm1d(num_hidden)
        self.batchNorm2 = nn.BatchNorm1d(hid2)
        self.batchNorm3 = nn.BatchNorm1d(hid3)
        self.batchNorm4 = nn.BatchNorm1d(hid4)
        self.batchNorm5 = nn.BatchNorm1d(hid5)
        self.dropout    = nn.Dropout(p=0.48)
        self.relu       = nn.ReLU()
        self.sigmoid    = nn.Sigmoid()
        self.out        = nn.Linear(hid5, 1)

    def forward(self, x):
        l1   = self.linear1(x)
        n1   = self.batchNorm1(l1)
        d1   = self.dropout(n1)
        act1 = self.relu(d1)
        l2   = self.linear2(act1)
        n2   = self.batchNorm2(l2)
        d2   = self.dropout(n2)
        act2 = self.relu(d2)
        l3   = self.linear3(act2)
        n3   = self.batchNorm3(l3)
        d3   = self.dropout(n3)
        act3 = self.relu(d3)
        l4   = self.linear4(act3)
        n4   = self.batchNorm4(l4)
        d4   = self.dropout(n4)
        act4 = self.relu(d4)
        l5   = self.linear5(act4)
        n5   = self.batchNorm5(l5)
        d5   = self.dropout(n5)
        act5 = self.relu(d5)
        l6   = self.out(act5)
        output = l6 #self.sigmoid(l6)
        return output
    

In [ ]:
class RMSELoss(torch.nn.Module):
    def init(self):
        super(RMSELoss,self).init()

    def forward(self,x,y):
        criterion = nn.MSELoss()
        loss = torch.sqrt(criterion(x, y))
        return loss

In [ ]:
loss = RMSELoss()
model = NeuralNetwork(inp,hid)

In [ ]:
def torch_fit(x, y, x_val, y_val,model, loss, lr, num_epochs):
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9,nesterov=False)#torch.optim.Adam(model.parameters(), lr=lr) # #torch.optim.RMSprop(model.parameters(), lr=lr, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)#torch.optim.Adagrad(model.parameters(), lr=lr, lr_decay=0, weight_decay=0, initial_accumulator_value=0, eps=1e-10)#torch.optim.Adadelta(model.parameters(), rho=0.9, eps=1e-06, weight_decay=0)# # #torch.optim.Adam(model.parameters(), lr=lr)
    count = -1
    pvl = 0
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        y_pred_tensor = model(x)
        loss_value = loss(y_pred_tensor, y)
        val_loss   = loss(model(x_val),y_val)
        print(f'Epoch {epoch}, loss {loss_value.item():.4f} val loss: {val_loss.item():.4f} count: {count}')
        loss_value.backward()
        optimizer.step()
        if val_loss.item() > pvl:
            count += 1
        pvl = val_loss
        if count > 3750: 
            break
    return model

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x = pre_pipe(df).values
x_train, x_val, y_train, y_val = train_test_split(x,y)
l = len(y_train)
lv = len(y_val)
l,lv

In [ ]:
x_tensor = torch.tensor(x_train).float().cuda()
x_val_tensor = torch.tensor(x_val).float().cuda()
y_tensor = torch.tensor(y_train.values).float().cuda()
y_val_tensor = torch.tensor(y_val.values).float().cuda()
y_tensor = y_tensor.view(l,1).cuda()
y_val_tensor = y_val_tensor.view(lv,1).cuda()

In [ ]:
x_tensor.size(), y_tensor.size()

In [ ]:
model.cuda()
nn_model = torch_fit(x_tensor, y_tensor, x_val_tensor, y_val_tensor, model=model, loss=loss, lr=0.00728, num_epochs=9550)

In [ ]:
s_df = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/sample_submission.csv')
s_df.head()
#/kaggle/working/subs.csv

In [ ]:
tes_tensor = torch.tensor(pre_pipe(test, False).values).float().cuda()
tes_tensor.size()

In [ ]:
preds = model(tes_tensor)
preds = preds.detach().cpu().numpy()

In [ ]:
s_df.target = preds 
s_df.head()

In [ ]:
s_df.to_csv('/kaggle/working/subs4.csv', index=False)